In [2]:
import yfinance as yf
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import datetime

# Function to fetch stock data
def fetch_data(symbol, start_date, end_date):
    data = yf.download(symbol, start=start_date, end=end_date)
    return data

# Function to fetch options data
def get_options_chain(symbol):
    stock = yf.Ticker(symbol)
    options = stock.option_chain()
    return options.calls, options.puts

# Function to calculate moving averages and predict next day prices
def prepare_data(data):
    data['SMA_20'] = data['Adj Close'].rolling(window=20).mean()
    data['SMA_50'] = data['Adj Close'].rolling(window=50).mean()
    data['Target'] = data['Adj Close'].shift(-1)
    data.dropna(inplace=True)
    return data

# Build and train the model
def train_model(data):
    features = ['Adj Close', 'SMA_20', 'SMA_50']
    X = data[features]
    y = data['Target']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    mse = mean_squared_error(y_test, model.predict(X_test))
    print(f"Model Mean Squared Error: {mse}")
    return model

# Analyze options and recommend which call to sell
def recommend_option_to_sell(calls, model, current_price):
    target_price = current_price * 1.05  # 5% above the current price as target
    calls['distance'] = abs(calls['strike'] - target_price)
    suitable_calls = calls[(calls['strike'] > current_price) & (calls['lastPrice'] > 0)]
    if not suitable_calls.empty:
        best_option = suitable_calls.loc[suitable_calls['distance'].idxmin()]
        print(f"Recommended Call to Sell: Strike {best_option['strike']}, Expiry {best_option['lastTradeDate']}, Premium {best_option['lastPrice']}")
    else:
        print("No suitable calls to recommend.")

# Main script execution
if __name__ == "__main__":
    symbol = 'AMZN'
    data = fetch_data(symbol, '2020-01-01', '2024-05-20')
    data = prepare_data(data)
    model = train_model(data)
    current_price = data.iloc[-1]['Adj Close']
    
    calls, puts = get_options_chain(symbol)
    recommend_option_to_sell(calls, model, current_price)


[*********************100%%**********************]  1 of 1 completed


Model Mean Squared Error: 9.917211112700356
Recommended Call to Sell: Strike 192.5, Expiry 2024-05-17 19:59:59+00:00, Premium 0.26


In [5]:
import yfinance as yf
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import datetime

# Function to fetch stock data
def fetch_data(symbol, start_date, end_date):
    data = yf.download(symbol, start=start_date, end=end_date)
    return data

# Function to fetch options data
def get_options_chain(symbol):
    stock = yf.Ticker(symbol)
    exp_dates = stock.options  # Get all available expiration dates
    all_calls = pd.DataFrame()
    for date in exp_dates:
        opt = stock.option_chain(date)
        calls = opt.calls
        calls['expiry'] = date  # Add expiration date to the DataFrame
        all_calls = pd.concat([all_calls, calls])
    return all_calls

# Function to calculate moving averages and predict next day prices
def prepare_data(data):
    data['SMA_20'] = data['Adj Close'].rolling(window=20).mean()
    data['SMA_50'] = data['Adj Close'].rolling(window=50).mean()
    data['Target'] = data['Adj Close'].shift(-1)
    data.dropna(inplace=True)
    return data

# Build and train the model
def train_model(data):
    features = ['Adj Close', 'SMA_20', 'SMA_50']
    X = data[features]
    y = data['Target']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    mse = mean_squared_error(y_test, model.predict(X_test))
    print(f"Model Mean Squared Error: {mse}")
    return model

import datetime

# In your function recommend_options_by_duration, modify the calculation of expiration_date:
def recommend_options_by_duration(calls, current_price):
    target_price_increase = 1.05  # Target strike price is 5% above current price
    durations = {'1 week': 7, '1 month': 30, '3 months': 90}
    for label, days in durations.items():
        expiration_date = pd.Timestamp(datetime.datetime.now() + datetime.timedelta(days=days))
        suitable_calls = calls[(calls['strike'] > current_price * target_price_increase) &
                               (calls['lastPrice'] > 0) &
                               (pd.to_datetime(calls['expiry']) <= expiration_date)]
        print(f"\nOptions to sell for {label}:")
        if not suitable_calls.empty:
            display(suitable_calls[['strike', 'expiry', 'lastPrice']].sort_values(by='lastPrice', ascending=False).head())
        else:
            print("No suitable calls found for this duration.")


# Main script execution
if __name__ == "__main__":
    symbol = 'AMZN'
    data = fetch_data(symbol, '2020-01-01', '2024-05-20')
    data = prepare_data(data)
    model = train_model(data)
    current_price = data.iloc[-1]['Adj Close']
    
    all_calls = get_options_chain(symbol)
    recommend_options_by_duration(all_calls, current_price)


[*********************100%%**********************]  1 of 1 completed


Model Mean Squared Error: 9.917211112700356

Options to sell for 1 week:


,strike,expiry,lastPrice
27,195.0,2024-05-24,0.13
28,197.5,2024-05-24,0.06
31,205.0,2024-05-24,0.04
41,245.0,2024-05-24,0.04
29,200.0,2024-05-24,0.03



Options to sell for 1 month:


,strike,expiry,lastPrice
19,195.0,2024-06-14,1.29
18,195.0,2024-06-07,0.82
20,200.0,2024-06-14,0.60
26,195.0,2024-05-31,0.38
19,200.0,2024-06-07,0.32



Options to sell for 3 months:


,strike,expiry,lastPrice
116,1080.0,2024-06-21,1475.05
120,1220.0,2024-06-21,1359.00
150,1840.0,2024-06-21,1223.65
117,1100.0,2024-06-21,1200.00
124,1300.0,2024-06-21,1175.80


In [14]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import datetime

# Function to fetch stock data
def fetch_data(symbol, start_date, end_date):
    data = yf.download(symbol, start=start_date, end=end_date, auto_adjust=True)  # Ensuring split adjustments
    return data


# Verify current stock price
def verify_current_price(symbol):
    stock = yf.Ticker(symbol)
    current_price = stock.history(period="1d")['Close'].iloc[-1]
    print(f"Current price of {symbol}: ${current_price}")
    return current_price


# Function to fetch options data
def get_options_chain(symbol):
    stock = yf.Ticker(symbol)
    exp_dates = stock.options  # Get all available expiration dates
    all_calls = pd.DataFrame()
    for date in exp_dates:
        opt = stock.option_chain(date)
        calls = opt.calls
        calls['expiry'] = date  # Add expiration date to the DataFrame
        all_calls = pd.concat([all_calls, calls])
    return all_calls

# Calculate historical volatility
def calculate_historical_volatility(data, days=252):
    data['Log_Return'] = np.log(data['Close'] / data['Close'].shift(1))
    hv = data['Log_Return'].std() * np.sqrt(days)  # Annualized Volatility
    return hv

# Function to calculate moving averages and predict next day prices
def prepare_data(data):
    data['SMA_20'] = data['Close'].rolling(window=20).mean()
    data['SMA_50'] = data['Close'].rolling(window=50).mean()
    data['Target'] = data['Close'].shift(-1)
    data.dropna(inplace=True)
    return data

# Build and train the model
def train_model(data):
    features = ['Close', 'SMA_20', 'SMA_50']
    X = data[features]
    y = data['Target']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    mse = mean_squared_error(y_test, model.predict(X_test))
    print(f"Model Mean Squared Error: {mse}")
    return model

# Recommend options based on historical volatility and durations
def recommend_options_by_duration(calls, current_price, hv):
    target_price_increase = 1 + hv  # Adjust the target based on historical volatility
    durations = {'1 week': 7, '1 month': 30, '3 months': 90}
    for label, days in durations.items():
        expiration_date = pd.Timestamp(datetime.datetime.now() + datetime.timedelta(days=days))
        suitable_calls = calls[(calls['strike'] > current_price * target_price_increase) &
                               (calls['lastPrice'] > 0) &
                               (pd.to_datetime(calls['expiry']) <= expiration_date)]
        print(f"\nOptions to sell for {label}:")
        if not suitable_calls.empty:
            print(suitable_calls[['strike', 'expiry', 'lastPrice']].sort_values(by='lastPrice', ascending=False).head())
        else:
            print("No suitable calls found for this duration.")

# Main execution to check current price
if __name__ == "__main__":
    symbol = 'AMZN'
    current_price = verify_current_price(symbol)  # This should show the correct current price
    data = fetch_data(symbol, '2023-01-01', '2024-05-17')
    data_prepared = prepare_data(data)
    model = train_model(data_prepared)
    current_price = data.iloc[-1]['Close']
    
    hv = calculate_historical_volatility(data)
    
    all_calls = get_options_chain(symbol)
    recommend_options_by_duration(all_calls, current_price, hv)


[*********************100%%**********************]  1 of 1 completed

Current price of AMZN: $184.6999969482422
Model Mean Squared Error: 8.54634907645365



Options to sell for 1 week:
    strike      expiry  lastPrice
41   245.0  2024-05-24       0.04
42   250.0  2024-05-24       0.01
43   255.0  2024-05-24       0.01

Options to sell for 1 month:
    strike      expiry  lastPrice
29   250.0  2024-06-07       0.09
41   245.0  2024-05-24       0.04
42   250.0  2024-05-24       0.01
43   255.0  2024-05-24       0.01
41   250.0  2024-05-31       0.01

Options to sell for 3 months:
     strike      expiry  lastPrice
116  1080.0  2024-06-21    1475.05
120  1220.0  2024-06-21    1359.00
150  1840.0  2024-06-21    1223.65
117  1100.0  2024-06-21    1200.00
124  1300.0  2024-06-21    1175.80


In [17]:
import yfinance as yf
import numpy as np
from scipy.stats import norm

# Fetch data
def fetch_data(symbol):
    stock = yf.Ticker('AMZN')
    return stock

# Calculate Black-Scholes Delta for call options
def black_scholes_delta(S, K, T, r, sigma, option_type="call"):
    d1 = (np.log(S / K) + (r + 0.5 * 0.24**2) * T) / (0.24 * np.sqrt(T))
    if option_type == "call":
        return norm.cdf(d1)
    else:
        return norm.cdf(d1) - 1

# Choose the option
def choose_option(stock, target_delta=0.48):
    options = stock.option_chain(stock.options[0]).calls
    options['delta'] = options.apply(lambda x: black_scholes_delta(
        S=stock.info['regularMarketPrice'],
        K=x['strike'],
        T=x['daysToExpiration'] / 365,
        r=0.01,  # Assuming a risk-free rate of 1%
        sigma=x['impliedVolatility']),
        axis=1)
    suitable_options = options[(options['delta'] < target_delta) & (options['delta'] > target_delta - 0.05)]
    return suitable_options

# Main function
if __name__ == "__main__":
    symbol = 'AMZN'
    stock = fetch_data(symbol)
    selected_options = choose_option(stock)
    print(selected_options[['strike', 'lastPrice', 'delta', 'expiry']])


KeyError: 'regularMarketPrice'

In [19]:
import pandas as pd
import yfinance as yf
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import datetime

# Function definitions as previously provided...

def calculate_earnings_and_risk(options, num_options=100):
    if not options.empty:
        total_premium = options['lastPrice'].iloc[0] * num_options * 100  # Assuming each option contract is for 100 shares
        risk_percentage = options['delta'].iloc[0]  # Assuming delta approximates the risk of expiring in-the-money
        return total_premium, risk_percentage
    return 0, 0

# Main script execution
if __name__ == "__main__":
    symbol = 'AMZN'
    data = fetch_data(symbol, '2020-01-01', '2024-05-20')
    prepared_data = prepare_data(data)
    model = train_model(prepared_data)
    current_price = data['Adj Close'].iloc[-1]
    
    all_calls = get_options_chain(symbol)
    suitable_calls = recommend_options_by_duration(all_calls, current_price)
    
    if suitable_calls is not None and not suitable_calls.empty:
        total_premium, risk_percentage = calculate_earnings_and_risk(suitable_calls)
        print(f"Total potential premium: ${total_premium:.2f}")
        print(f"Risk of expiring in-the-money: {risk_percentage*100:.2f}%")
    else:
        print("No suitable options found for selling.")


TypeError: fetch_data() takes 1 positional argument but 3 were given

In [21]:
import pandas as pd
import yfinance as yf
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import datetime
import logging

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Function to fetch stock data
def fetch_data(symbol, start_date, end_date):
    try:
        data = yf.download(symbol, start=start_date, end=end_date)
        logging.info("Data fetched successfully")
        return data
    except Exception as e:
        logging.error(f"Error fetching data: {e}")
        return pd.DataFrame()

# Function to fetch options data
def get_options_chain(symbol):
    try:
        stock = yf.Ticker(symbol)
        exp_dates = stock.options  # Get all available expiration dates
        all_calls = pd.DataFrame()
        for date in exp_dates:
            opt = stock.option_chain(date)
            calls = opt.calls
            calls['expiry'] = date  # Add expiration date to the DataFrame
            all_calls = pd.concat([all_calls, calls])
        logging.info("Options data fetched successfully")
        return all_calls
    except Exception as e:
        logging.error(f"Error fetching options data: {e}")
        return pd.DataFrame()

# Function to calculate moving averages and predict next day prices
def prepare_data(data):
    if data.empty:
        return data
    data['SMA_20'] = data['Adj Close'].rolling(window=20).mean()
    data['SMA_50'] = data['Adj Close'].rolling(window=50).mean()
    data['Target'] = data['Adj Close'].shift(-1)
    data.dropna(inplace=True)
    return data

# Build and train the model
def train_model(data):
    if data.empty:
        return None
    features = ['Adj Close', 'SMA_20', 'SMA_50']
    X = data[features]
    y = data['Target']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    mse = mean_squared_error(y_test, model.predict(X_test))
    logging.info(f"Model trained successfully with MSE: {mse}")
    return model

# In your function recommend_options_by_duration, modify the calculation of expiration_date:
def recommend_options_by_duration(calls, current_price):
    if calls.empty:
        return pd.DataFrame()
    target_price_increase = 1.05  # Input details for target price increase percentage
    durations = {'1 week': 7, '1 month': 30, '3 months': 90}  # Input details for durations
    suitable_calls = pd.DataFrame()
    for label, days in durations.items():
        expiration_date = pd.Timestamp(datetime.datetime.now() + datetime.timedelta(days=days))
        filtered_calls = calls[(calls['strike'] > current_price * target_price_increase) &
                               (calls['lastPrice'] > 0) &
                               (pd.to_datetime(calls['expiry']) <= expiration_date)]
        if not filtered_calls.empty:
            suitable_calls = pd.concat([suitable_calls, filtered_calls])
        logging.info(f"Options to sell for {label} fetched")
    return suitable_calls

# Calculate earnings and risk from selected options
def calculate_earnings_and_risk(options, num_options=100):  # Input details for number of options
    if options.empty:
        return 0, 0
    total_premium = options['lastPrice'].iloc[0] * num_options * 100  # Assuming each option contract is for 100 shares
    risk_percentage = options['delta'].iloc[0]  # Using delta as risk, hypothetical here
    return total_premium, risk_percentage

# Main script execution
if __name__ == "__main__":
    symbol = 'AMZN'  # Input details for symbol
    data = fetch_data(symbol, '2023-01-01', '2024-05-20')  # Input details for date range
    prepared_data = prepare_data(data)
    model = train_model(prepared_data)
    if model:
        current_price = data['Adj Close'].iloc[-1]
        all_calls = get_options_chain(symbol)
        suitable_calls = recommend_options_by_duration(all_calls, current_price)
        if not suitable_calls.empty:
            total_premium, risk_percentage = calculate_earnings_and_risk(suitable_calls)
            print(f"Total potential premium: ${total_premium:.2f}")
            print(f"Risk of expiring in-the-money: {risk_percentage*100:.2f}%")
        else:
            print("No suitable options found based on the criteria.")


[*********************100%%**********************]  1 of 1 completed
2024-05-20 16:12:08,231 - INFO - Data fetched successfully


2024-05-20 16:12:08,438 - INFO - Model trained successfully with MSE: 10.546790328560348
2024-05-20 16:12:17,504 - INFO - Options data fetched successfully
2024-05-20 16:12:17,508 - INFO - Options to sell for 1 week fetched
2024-05-20 16:12:17,513 - INFO - Options to sell for 1 month fetched
2024-05-20 16:12:17,518 - INFO - Options to sell for 3 months fetched


KeyError: 'delta'

In [22]:
import pandas as pd
import yfinance as yf
import numpy as np
from scipy.stats import norm
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import datetime
import logging

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Function to fetch stock data
def fetch_data(symbol, start_date, end_date):
    try:
        data = yf.download(symbol, start=start_date, end=end_date)
        logging.info("Data fetched successfully")
        return data
    except Exception as e:
        logging.error(f"Error fetching data: {e}")
        return pd.DataFrame()

# Function to fetch options data
def get_options_chain(symbol):
    try:
        stock = yf.Ticker(symbol)
        exp_dates = stock.options  # Get all available expiration dates
        all_calls = pd.DataFrame()
        for date in exp_dates:
            opt = stock.option_chain(date)
            calls = opt.calls
            calls['expiry'] = date  # Add expiration date to the DataFrame
            all_calls = pd.concat([all_calls, calls])
        logging.info("Options data fetched successfully")
        return all_calls
    except Exception as e:
        logging.error(f"Error fetching options data: {e}")
        return pd.DataFrame()

# Function to calculate moving averages and predict next day prices
def prepare_data(data):
    if data.empty:
        return data
    data['SMA_20'] = data['Adj Close'].rolling(window=20).mean()
    data['SMA_50'] = data['Adj Close'].rolling(window=50).mean()
    data['Target'] = data['Adj Close'].shift(-1)
    data.dropna(inplace=True)
    return data

# Build and train the model
def train_model(data):
    if data.empty:
        return None
    features = ['Adj Close', 'SMA_20', 'SMA_50']
    X = data[features]
    y = data['Target']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    mse = mean_squared_error(y_test, model.predict(X_test))
    logging.info(f"Model trained successfully with MSE: {mse}")
    return model

# Calculate Black-Scholes Delta for call options
def black_scholes_delta(S, K, T, r, sigma, option_type="call"):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    if option_type == "call":
        return norm.cdf(d1)
    else:
        return norm.cdf(d1) - 1

# Recommend options by duration with Delta calculation
def recommend_options_by_duration(calls, current_price, iv, interest_rate=0.01):
    if calls.empty:
        return pd.DataFrame()
    target_price_increase = 1.05  # Input details for target price increase percentage
    durations = {'1 week': 7, '1 month': 30, '3 months': 90}  # Input details for durations
    suitable_calls = pd.DataFrame()
    for label, days in durations.items():
        expiration_date = pd.Timestamp(datetime.datetime.now() + datetime.timedelta(days=days))
        filtered_calls = calls[(calls['strike'] > current_price * target_price_increase) &
                               (calls['lastPrice'] > 0) &
                               (pd.to_datetime(calls['expiry']) <= expiration_date)]
        if not filtered_calls.empty:
            filtered_calls['daysToExpiration'] = (pd.to_datetime(filtered_calls['expiry']) - datetime.datetime.now()).dt.days
            filtered_calls['delta'] = filtered_calls.apply(lambda x: black_scholes_delta(
                S=current_price,
                K=x['strike'],
                T=x['daysToExpiration'] / 365,
                r=interest_rate,
                sigma=iv), axis=1)
            suitable_calls = pd.concat([suitable_calls, filtered_calls])
        logging.info(f"Options to sell for {label} fetched")
    return suitable_calls

# Calculate earnings and risk from selected options
def calculate_earnings_and_risk(options, num_options=100):  # Input details for number of options
    if options.empty:
        return 0, 0
    total_premium = options['lastPrice'].iloc[0] * num_options * 100  # Assuming each option contract is for 100 shares
    risk_percentage = options['delta'].iloc[0]  # Using delta as risk, hypothetical here
    return total_premium, risk_percentage

# Main script execution
if __name__ == "__main__":
    symbol = 'AMZN'  # Input details for symbol
    data = fetch_data(symbol, '2020-01-01', '2024-05-20')  # Input details for date range
    prepared_data = prepare_data(data)
    model = train_model(prepared_data)
    if model:
        current_price = data['Adj Close'].iloc[-1]
        all_calls = get_options_chain(symbol)
        iv = 0.2420  # Example placeholder value for implied volatility
        suitable_calls = recommend_options_by_duration(all_calls, current_price, iv)
        if not suitable_calls.empty:
            total_premium, risk_percentage = calculate_earnings_and_risk(suitable_calls)
            print(f"Total potential premium: ${total_premium:.2f}")
            print(f"Risk of expiring in-the-money: {risk_percentage*100:.2f}%")
        else:
            print("No suitable options found based on the criteria.")


[*********************100%%**********************]  1 of 1 completed
2024-05-20 16:15:19,583 - INFO - Data fetched successfully
2024-05-20 16:15:19,921 - INFO - Model trained successfully with MSE: 9.917211112700356
2024-05-20 16:15:29,713 - INFO - Options data fetched successfully
/var/folders/3x/j0kbtrs11j11nc47ldqz7vs00000gn/T/ipykernel_7770/2426641260.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_calls['daysToExpiration'] = (pd.to_datetime(filtered_calls['expiry']) - datetime.datetime.now()).dt.days
/var/folders/3x/j0kbtrs11j11nc47ldqz7vs00000gn/T/ipykernel_7770/2426641260.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



Total potential premium: $1300.00
Risk of expiring in-the-money: 0.32%


In [24]:
import pandas as pd
import yfinance as yf
import numpy as np
from scipy.stats import norm
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import datetime
import logging

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Function to fetch stock data
def fetch_data(symbol, start_date, end_date):
    try:
        data = yf.download(symbol, start=start_date, end=end_date)
        logging.info("Data fetched successfully")
        return data
    except Exception as e:
        logging.error(f"Error fetching data: {e}")
        return pd.DataFrame()

# Function to fetch options data
def get_options_chain(symbol):
    try:
        stock = yf.Ticker(symbol)
        exp_dates = stock.options  # Get all available expiration dates
        all_calls = pd.DataFrame()
        for date in exp_dates:
            opt = stock.option_chain(date)
            calls = opt.calls
            calls['expiry'] = date  # Add expiration date to the DataFrame
            all_calls = pd.concat([all_calls, calls])
        logging.info("Options data fetched successfully")
        return all_calls
    except Exception as e:
        logging.error(f"Error fetching options data: {e}")
        return pd.DataFrame()

# Function to calculate moving averages and predict next day prices
def prepare_data(data):
    if data.empty:
        return data
    data['SMA_20'] = data['Adj Close'].rolling(window=20).mean()
    data['SMA_50'] = data['Adj Close'].rolling(window=50).mean()
    data['Target'] = data['Adj Close'].shift(-1)
    data.dropna(inplace=True)
    return data

# Build and train the model
def train_model(data):
    if data.empty:
        return None
    features = ['Adj Close', 'SMA_20', 'SMA_50']
    X = data[features]
    y = data['Target']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    mse = mean_squared_error(y_test, model.predict(X_test))
    logging.info(f"Model trained successfully with MSE: {mse}")
    return model

# Calculate Black-Scholes Delta for call options
def black_scholes_delta(S, K, T, r, sigma, option_type="call"):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    if option_type == "call":
        return norm.cdf(d1)
    else:
        return norm.cdf(d1) - 1

# Recommend options by duration with Delta calculation
def recommend_options_by_duration(calls, current_price, iv, interest_rate=0.01):
    if calls.empty:
        return pd.DataFrame()
    target_price_increase = 1.05  # Input details for target price increase percentage
    durations = {'1 week': 7, '1 month': 30, '3 months': 90}  # Input details for durations
    suitable_calls = pd.DataFrame()
    for label, days in durations.items():
        expiration_date = pd.Timestamp(datetime.datetime.now() + datetime.timedelta(days=days))
        filtered_calls = calls[(calls['strike'] > current_price * target_price_increase) &
                               (calls['lastPrice'] > 0) &
                               (pd.to_datetime(calls['expiry']) <= expiration_date)]
        if not filtered_calls.empty:
            filtered_calls['daysToExpiration'] = (pd.to_datetime(filtered_calls['expiry']) - datetime.datetime.now()).dt.days
            filtered_calls['delta'] = filtered_calls.apply(lambda x: black_scholes_delta(
                S=current_price,
                K=x['strike'],
                T=x['daysToExpiration'] / 365,
                r=interest_rate,
                sigma=iv), axis=1)
            suitable_calls = pd.concat([suitable_calls, filtered_calls])
        logging.info(f"Options to sell for {label} fetched")
    return suitable_calls

# Calculate earnings and risk from selected options
def calculate_earnings_and_risk(options, num_options=100):  # Input details for number of options
    if options.empty:
        return 0, 0
    total_premium = options['lastPrice'].iloc[0] * num_options * 100  # Assuming each option contract is for 100 shares
    risk_percentage = options['delta'].iloc[0]  # Using delta as risk, hypothetical here
    return total_premium, risk_percentage

# Main script execution
if __name__ == "__main__":
    symbol = 'AMZN'  # Input details for symbol
    data = fetch_data(symbol, '2023-01-01', '2024-05-20')  # Input details for date range
    prepared_data = prepare_data(data)
    model = train_model(prepared_data)
    if model:
        current_price = data['Adj Close'].iloc[-1]
        all_calls = get_options_chain(symbol)
        iv = 0.2420  # Example placeholder value for implied volatility
        suitable_calls = recommend_options_by_duration(all_calls, current_price, iv)
        if not suitable_calls.empty:
            total_premium, risk_percentage = calculate_earnings_and_risk(suitable_calls)
            print(f"Total potential premium: ${total_premium:.2f}")
            print(f"Risk of expiring in-the-money: {risk_percentage*100:.2f}%")
            # Show the best options to consider
            print("Recommended Options to Check Out:")
            print(suitable_calls[['strike', 'expiry', 'lastPrice', 'delta']].sort_values(by='lastPrice', ascending=False).head())
        else:
            print("No suitable options found based on the criteria.")


[*********************100%%**********************]  1 of 1 completed


2024-05-20 16:18:50,808 - INFO - Data fetched successfully
2024-05-20 16:18:51,034 - INFO - Model trained successfully with MSE: 10.546790328560348
2024-05-20 16:18:55,883 - INFO - Options data fetched successfully
/var/folders/3x/j0kbtrs11j11nc47ldqz7vs00000gn/T/ipykernel_7770/934069208.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_calls['daysToExpiration'] = (pd.to_datetime(filtered_calls['expiry']) - datetime.datetime.now()).dt.days
/var/folders/3x/j0kbtrs11j11nc47ldqz7vs00000gn/T/ipykernel_7770/934069208.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

Total potential premium: $1300.00
Risk of expiring in-the-money: 0.32%
Recommended Options to Check Out:
     strike      expiry  lastPrice          delta
116  1080.0  2024-06-21    1475.05  4.634756e-139
120  1220.0  2024-06-21    1359.00  1.470186e-158
150  1840.0  2024-06-21    1223.65  7.717081e-234
117  1100.0  2024-06-21    1200.00  6.509434e-142
124  1300.0  2024-06-21    1175.80  3.117460e-169


In [25]:
import pandas as pd
import yfinance as yf
import numpy as np
from scipy.stats import norm
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import datetime
import logging

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Function to fetch stock data
def fetch_data(symbol, start_date, end_date):
    try:
        data = yf.download(symbol, start=start_date, end=end_date)
        logging.info("Data fetched successfully")
        return data
    except Exception as e:
        logging.error(f"Error fetching data: {e}")
        return pd.DataFrame()

# Function to fetch options data
def get_options_chain(symbol):
    try:
        stock = yf.Ticker(symbol)
        exp_dates = stock.options  # Get all available expiration dates
        all_calls = pd.DataFrame()
        for date in exp_dates:
            opt = stock.option_chain(date)
            calls = opt.calls
            calls['expiry'] = date  # Add expiration date to the DataFrame
            all_calls = pd.concat([all_calls, calls])
        logging.info("Options data fetched successfully")
        return all_calls
    except Exception as e:
        logging.error(f"Error fetching options data: {e}")
        return pd.DataFrame()

# Function to calculate moving averages and predict next day prices
def prepare_data(data):
    if data.empty:
        return data
    data['SMA_20'] = data['Adj Close'].rolling(window=20).mean()
    data['SMA_50'] = data['Adj Close'].rolling(window=50).mean()
    data['Target'] = data['Adj Close'].shift(-1)
    data.dropna(inplace=True)
    return data

# Build and train the model
def train_model(data):
    if data.empty:
        return None
    features = ['Adj Close', 'SMA_20', 'SMA_50']
    X = data[features]
    y = data['Target']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    mse = mean_squared_error(y_test, model.predict(X_test))
    logging.info(f"Model trained successfully with MSE: {mse}")
    return model

# Calculate Black-Scholes Delta for call options
def black_scholes_delta(S, K, T, r, sigma, option_type="call"):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    if option_type == "call":
        return norm.cdf(d1)
    else:
        return norm.cdf(d1) - 1

# Recommend options by duration with Delta calculation
def recommend_options_by_duration(calls, current_price, iv, interest_rate=0.01):
    if calls.empty:
        return pd.DataFrame()
    target_price_increase = 1.05  # Input details for target price increase percentage
    durations = {'1 week': 7, '1 month': 30, '3 months': 90}  # Input details for durations
    suitable_calls = pd.DataFrame()
    for label, days in durations.items():
        expiration_date = pd.Timestamp(datetime.datetime.now() + datetime.timedelta(days=days))
        filtered_calls = calls[(calls['strike'] > current_price * target_price_increase) &
                               (calls['strike'] <= current_price * 2) &  # Ensure strikes are within reasonable bounds
                               (calls['lastPrice'] > 0) &
                               (pd.to_datetime(calls['expiry']) <= expiration_date)]
        if not filtered_calls.empty:
            filtered_calls['daysToExpiration'] = (pd.to_datetime(filtered_calls['expiry']) - datetime.datetime.now()).dt.days
            filtered_calls['delta'] = filtered_calls.apply(lambda x: black_scholes_delta(
                S=current_price,
                K=x['strike'],
                T=x['daysToExpiration'] / 365,
                r=interest_rate,
                sigma=iv), axis=1)
            suitable_calls = pd.concat([suitable_calls, filtered_calls])
        logging.info(f"Options to sell for {label} fetched")
    return suitable_calls

# Calculate earnings and risk from selected options
def calculate_earnings_and_risk(options, num_options=100):  # Input details for number of options
    if options.empty:
        return 0, 0
    total_premium = options['lastPrice'].iloc[0] * num_options * 100  # Assuming each option contract is for 100 shares
    risk_percentage = options['delta'].iloc[0]  # Using delta as risk, hypothetical here
    return total_premium, risk_percentage

# Main script execution
if __name__ == "__main__":
    symbol = 'AMZN'  # Input details for symbol
    data = fetch_data(symbol, '2020-01-01', '2024-05-20')  # Input details for date range
    prepared_data = prepare_data(data)
    model = train_model(prepared_data)
    if model:
        current_price = data['Adj Close'].iloc[-1]
        all_calls = get_options_chain(symbol)
        iv = 0.2420  # Example placeholder value for implied volatility
        suitable_calls = recommend_options_by_duration(all_calls, current_price, iv)
        if not suitable_calls.empty:
            total_premium, risk_percentage = calculate_earnings_and_risk(suitable_calls)
            print(f"Total potential premium: ${total_premium:.2f}")
            print(f"Risk of expiring in-the-money: {risk_percentage*100:.2f}%")
            # Show the best options to consider
            print("Recommended Options to Check Out:")
            print(suitable_calls[['strike', 'expiry', 'lastPrice', 'delta']].sort_values(by='lastPrice', ascending=False).head())
        else:
            print("No suitable options found based on the criteria.")


[*********************100%%**********************]  1 of 1 completed
2024-05-20 16:21:55,823 - INFO - Data fetched successfully
2024-05-20 16:21:56,479 - INFO - Model trained successfully with MSE: 9.917211112700356
2024-05-20 16:22:06,306 - INFO - Options data fetched successfully
/var/folders/3x/j0kbtrs11j11nc47ldqz7vs00000gn/T/ipykernel_7770/1771435320.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_calls['daysToExpiration'] = (pd.to_datetime(filtered_calls['expiry']) - datetime.datetime.now()).dt.days
/var/folders/3x/j0kbtrs11j11nc47ldqz7vs00000gn/T/ipykernel_7770/1771435320.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



Total potential premium: $1300.00
Risk of expiring in-the-money: 0.32%
Recommended Options to Check Out:
    strike      expiry  lastPrice     delta
22   195.0  2024-08-16       6.97  0.333876
23   200.0  2024-08-16       5.35  0.259942
24   205.0  2024-08-16       3.85  0.196963
24   195.0  2024-07-19       3.38  0.290405
25   210.0  2024-08-16       2.90  0.145375


In [28]:
import pandas as pd
import yfinance as yf
import numpy as np
from scipy.stats import norm
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import datetime
import logging

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Function to fetch stock data
def fetch_data(symbol, start_date, end_date):
    try:
        data = yf.download(symbol, start=start_date, end=end_date)
        logging.info("Data fetched successfully")
        return data
    except Exception as e:
        logging.error(f"Error fetching data: {e}")
        return pd.DataFrame()

# Function to fetch options data
def get_options_chain(symbol):
    try:
        stock = yf.Ticker(symbol)
        exp_dates = stock.options  # Get all available expiration dates
        all_calls = pd.DataFrame()
        for date in exp_dates:
            opt = stock.option_chain(date)
            calls = opt.calls
            calls['expiry'] = date  # Add expiration date to the DataFrame
            all_calls = pd.concat([all_calls, calls])
        logging.info("Options data fetched successfully")
        return all_calls
    except Exception as e:
        logging.error(f"Error fetching options data: {e}")
        return pd.DataFrame()

# Function to calculate moving averages and predict next day prices
def prepare_data(data):
    if data.empty:
        return data
    data['SMA_20'] = data['Adj Close'].rolling(window=20).mean()
    data['SMA_50'] = data['Adj Close'].rolling(window=50).mean()
    data['Target'] = data['Adj Close'].shift(-1)
    data.dropna(inplace=True)
    return data

# Build and train the model
def train_model(data):
    if data.empty:
        return None
    features = ['Adj Close', 'SMA_20', 'SMA_50']
    X = data[features]
    y = data['Target']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    mse = mean_squared_error(y_test, model.predict(X_test))
    logging.info(f"Model trained successfully with MSE: {mse}")
    return model

# Calculate Black-Scholes Delta for call options
def black_scholes_delta(S, K, T, r, sigma, option_type="call"):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    if option_type == "call":
        return norm.cdf(d1)
    else:
        return norm.cdf(d1) - 1

# Recommend options by duration with Delta calculation
def recommend_options_by_duration(calls, current_price, iv, interest_rate=0.01):
    if calls.empty:
        return pd.DataFrame()
    target_price_increase = 1.07  # Input details for target price increase percentage
    durations = {'1 week': 7, '1 month': 30, '3 months': 90}  # Input details for durations
    suitable_calls = pd.DataFrame()
    for label, days in durations.items():
        expiration_date = pd.Timestamp(datetime.datetime.now() + datetime.timedelta(days=days))
        filtered_calls = calls[(calls['strike'] > current_price * target_price_increase) &
                               (calls['strike'] <= current_price * 2) &  # Ensure strikes are within reasonable bounds
                               (calls['lastPrice'] > 0) &
                               (pd.to_datetime(calls['expiry']) <= expiration_date)]
        if not filtered_calls.empty:
            filtered_calls['daysToExpiration'] = (pd.to_datetime(filtered_calls['expiry']) - datetime.datetime.now()).dt.days
            filtered_calls['delta'] = filtered_calls.apply(lambda x: black_scholes_delta(
                S=current_price,
                K=x['strike'],
                T=x['daysToExpiration'] / 365,
                r=interest_rate,
                sigma=iv), axis=1)
            filtered_calls['total_premium'] = filtered_calls['lastPrice'] * 100  # Assuming 1 contract = 100 shares
            filtered_calls['risk'] = filtered_calls['delta'] * 100  # Risk in percentage
            suitable_calls = pd.concat([suitable_calls, filtered_calls])
        logging.info(f"Options to sell for {label} fetched")
    return suitable_calls

# Main script execution
if __name__ == "__main__":
    symbol = 'AMZN'  # Input details for symbol
    data = fetch_data(symbol, '2020-01-01', '2024-05-20')  # Input details for date range
    prepared_data = prepare_data(data)
    model = train_model(prepared_data)
    if model:
        current_price = data['Adj Close'].iloc[-1]
        all_calls = get_options_chain(symbol)
        iv = 0.2344  # Example placeholder value for implied volatility
        suitable_calls = recommend_options_by_duration(all_calls, current_price, iv)
        if not suitable_calls.empty:
            print("Recommended Options to Check Out:")
            print(suitable_calls[['strike', 'expiry', 'lastPrice', 'delta', 'total_premium', 'risk']].sort_values(by='lastPrice', ascending=False).head())
        else:
            print("No suitable options found based on the criteria.")


[*********************100%%**********************]  1 of 1 completed


2024-05-20 16:33:08,340 - INFO - Data fetched successfully
2024-05-20 16:33:08,820 - INFO - Model trained successfully with MSE: 9.917211112700356
2024-05-20 16:33:18,069 - INFO - Options data fetched successfully
/var/folders/3x/j0kbtrs11j11nc47ldqz7vs00000gn/T/ipykernel_7770/3566794999.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_calls['daysToExpiration'] = (pd.to_datetime(filtered_calls['expiry']) - datetime.datetime.now()).dt.days
/var/folders/3x/j0kbtrs11j11nc47ldqz7vs00000gn/T/ipykernel_7770/3566794999.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

Recommended Options to Check Out:
    strike      expiry  lastPrice     delta  total_premium       risk
23   200.0  2024-08-16       5.35  0.252016          535.0  25.201606
24   205.0  2024-08-16       3.85  0.188367          385.0  18.836659
25   210.0  2024-08-16       2.90  0.136867          290.0  13.686694
25   200.0  2024-07-19       2.20  0.199933          220.0  19.993264
26   215.0  2024-08-16       1.98  0.096780          198.0   9.677960
